In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from pycaret.classification import *


In [2]:
#データの読み込みと前処理
import pandas as pd
train_data = pd.read_csv("./csv/train_data.csv", index_col='id')
test_data = pd.read_csv("./csv/test_data.csv", index_col='id')
print(train_data.shape,test_data.shape)

(8878, 2) (24838, 1)


In [3]:
np.sum(train_data['y'] == 0), np.sum(train_data['y'] == 1)


(8707, 171)

In [4]:
X = train_data['contents']
y = train_data['y']

In [16]:
#scikit-learnライブラリをimport
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV,StratifiedKFold
from lightgbm import LGBMClassifier
# 回帰モデルのインスタンス化
tfidf = TfidfVectorizer()
tfidf.fit(X)
vec_X = tfidf.transform(X)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(
    vec_X, y, test_size=0.1, stratify=y)

In [21]:
model = LGBMClassifier()

LGBMClassifier()

In [23]:
grid_param ={
    'n_estimators':[1000,2000],
    'max_depth':[4,8,16],
    'num_leaves':[31,15,7,3],
    'learning_rate':[0.1,0.05,0.01]
}

fit_params={'early_stopping_rounds':10, 
            'eval_metric' : 'auc', 
            'eval_set' : [(X_test, y_test)]
           }
grid = GridSearchCV(
            model, 
            grid_param, 
            cv = StratifiedKFold(n_splits=3, shuffle=True), # 交差検定の回数
            scoring = 'f1',
            verbose = 0
            )

grid.fit(
            X_test, 
            y_test,
            **fit_params,
            verbose = 0
            )

[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=True),
             estimator=LGBMClassifier(),
             param_grid={'learning_rate': [0.1, 0.05, 0.01],
                         'max_depth': [4, 8, 16], 'n_estimators': [1000, 2000],
                         'num_leaves': [31, 15, 7, 3]},
             scoring='f1')

In [24]:
from sklearn.metrics import confusion_matrix, f1_score
print(confusion_matrix(y, model.predict(vec_X)))

[[8707    0]
 [  12  159]]


In [25]:
print(f1_score(y, model.predict(vec_X))) #ほとんど検出できず

0.9636363636363636


In [29]:
test_X = tfidf.transform(test_data['contents'])

In [31]:
best_params = grid.best_params_
test_predicted = grid.predict(test_X)

In [ ]:
#test_predicted = model.predict(test_X)

In [32]:

submit_df = pd.DataFrame({'y': test_predicted})
submit_df.index.name = 'id'
submit_df.index += 1
submit_df.to_csv('./sub/stratified_cv_lgbm_submission.csv')